# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [2]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [3]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {'table': ['orders', 'customers', 'products', 'suppliers'],
        'definition': ['orders: orders table with order_id, customer_id, order_date, total_amount', 'customers: customers table with customer_id, name, email, phone', 'products: products table with product_id, name, price, stock', 'suppliers: suppliers table with supplier_id, name, contact_name, phone']}
df = pd.DataFrame(data)
print(df)

       table                                         definition
0     orders  orders: orders table with order_id, customer_i...
1  customers  customers: customers table with customer_id, n...
2   products  products: products table with product_id, name...
3  suppliers  suppliers: suppliers table with supplier_id, n...


In [4]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [5]:
print(text_tables)

orders: orders: orders table with order_id, customer_id, order_date, total_amount
customers: customers: customers table with customer_id, name, email, phone
products: products: products table with product_id, name, price, stock
suppliers: suppliers: suppliers table with supplier_id, name, contact_name, phone


In [6]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [7]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question="What is the total amount of orders for each customer?")

In [8]:
print(return_OAI(pqt1))

{
    "tables": ["orders", "customers"]
}


In [10]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question="What is the total amount of orders for each customer?")

In [11]:
print(return_OAI(pqt3))

{
    "tables": ["orders", "customers"]
}


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [19]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Cargar variables de entorno
load_dotenv()
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

def generate_sql(question):
    """
    Genera una consulta SQL basada en una pregunta en lenguaje natural
    """
    # Definir el contexto de la base de datos
    database_schema = """
    Tables and their columns:
    - orders (order_id, customer_id, order_date, total_amount)
    - customers (customer_id, name, email, segment)
    - products (product_id, name, category, price, stock_level)
    - order_items (order_id, product_id, quantity, unit_price)
    - inventory (product_id, quantity, last_restock_date, reorder_point)
    """
    
    prompt = f"""
    Given the following database schema:
    {database_schema}
    
    Generate a SQL query to answer this question:
    {question}
    
    Return only the SQL query, without any explanations.
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a SQL expert. Generate clear and efficient SQL queries."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        
        # Extraer la consulta SQL de la respuesta
        sql_query = response.choices[0].message.content.strip()
        return sql_query
        
    except Exception as e:
        return f"Error generating SQL: {str(e)}"

# Probar la función con una pregunta simple
test_question = "What are the top 5 customers by total order amount?"
print("Test Question:", test_question)
print("\nGenerated SQL:")
print(generate_sql(test_question))

Test Question: What are the top 5 customers by total order amount?

Generated SQL:
```sql
SELECT c.customer_id, c.name, SUM(o.total_amount) AS total_order_amount
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY total_order_amount DESC
LIMIT 5;
```


In [20]:
# Definir las preguntas
sales_questions = [
    "Show me the total sales by product category for the last month",
    "Which products have had the highest profit margin this year?",
    "List the top 10 customers by total purchase amount",
    "Calculate the monthly sales growth rate for each product"
]

# Probar las preguntas
print("=== Análisis de Ventas y Productos ===")
for question in sales_questions:
    print(f"\nPregunta: {question}")
    sql_query = generate_sql(question)
    print(f"SQL Generado:\n{sql_query}\n")
    print("-" * 50)

=== Análisis de Ventas y Productos ===

Pregunta: Show me the total sales by product category for the last month
SQL Generado:
```sql
SELECT p.category, SUM(oi.quantity * oi.unit_price) AS total_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
GROUP BY p.category;
```

--------------------------------------------------

Pregunta: Which products have had the highest profit margin this year?
SQL Generado:
```sql
WITH order_revenue AS (
    SELECT oi.product_id, SUM(oi.quantity * oi.unit_price) AS revenue
    FROM orders o
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE EXTRACT(YEAR FROM o.order_date) = EXTRACT(YEAR FROM CURRENT_DATE)
    GROUP BY oi.product_id
),
product_cost AS (
    SELECT p.product_id, SUM(i.quantity * p.price) AS cost
    FROM products p
    JOIN inventory i ON p.product_id = i.product_id
    GROUP BY p.product_id
)
SELECT p.pro

In [21]:
inventory_questions = [
    "Which products are currently out of stock?",
    "Show me products with inventory levels below reorder point",
    "Calculate the average time between restocks for each product",
    "List products with the highest inventory turnover rate"
]

print("=== Análisis de Inventario ===")
for question in inventory_questions:
    print(f"\nPregunta: {question}")
    sql_query = generate_sql(question)
    print(f"SQL Generado:\n{sql_query}\n")
    print("-" * 50)

=== Análisis de Inventario ===

Pregunta: Which products are currently out of stock?
SQL Generado:
```sql
SELECT name
FROM products
WHERE product_id NOT IN (
    SELECT product_id
    FROM inventory
    WHERE quantity > 0
);
```

--------------------------------------------------

Pregunta: Show me products with inventory levels below reorder point
SQL Generado:
```sql
SELECT p.product_id, p.name, i.quantity, i.reorder_point
FROM products p
JOIN inventory i ON p.product_id = i.product_id
WHERE i.quantity < i.reorder_point;
```

--------------------------------------------------

Pregunta: Calculate the average time between restocks for each product
SQL Generado:
```sql
SELECT product_id, AVG(DATEDIFF(day, last_restock_date, LAG(last_restock_date) OVER (PARTITION BY product_id ORDER BY last_restock_date))) AS avg_time_between_restocks
FROM inventory
GROUP BY product_id;
```

--------------------------------------------------

Pregunta: List products with the highest inventory turnover r

In [22]:
customer_questions = [
    "Find customers who haven't made a purchase in the last 3 months",
    "Calculate the customer lifetime value for our top 20 customers",
    "Show me the purchase frequency by customer segment",
    "Which customers have increased their spending by more than 20% compared to last year?"
]

print("=== Análisis de Clientes ===")
for question in customer_questions:
    print(f"\nPregunta: {question}")
    sql_query = generate_sql(question)
    print(f"SQL Generado:\n{sql_query}\n")
    print("-" * 50)

=== Análisis de Clientes ===

Pregunta: Find customers who haven't made a purchase in the last 3 months
SQL Generado:
```sql
SELECT c.customer_id, c.name
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id
WHERE o.order_date IS NULL OR o.order_date < DATE_SUB(NOW(), INTERVAL 3 MONTH)
```

--------------------------------------------------

Pregunta: Calculate the customer lifetime value for our top 20 customers
SQL Generado:
```sql
WITH customer_orders AS (
    SELECT c.customer_id, SUM(oi.quantity * oi.unit_price) AS total_spent
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    GROUP BY c.customer_id
),
customer_lifetime_value AS (
    SELECT co.customer_id, SUM(co.total_spent) AS lifetime_value
    FROM customer_orders co
    GROUP BY co.customer_id
    ORDER BY lifetime_value DESC
    LIMIT 20
)
SELECT clv.customer_id, clv.lifetime_value
FROM customer_lifetime_value clv;
```

-------------

In [23]:
def test_sql_generation(questions, category):
    """
    Función para probar la generación de SQL con análisis de resultados
    """
    results = []
    print(f"\n=== Testing {category} Questions ===\n")
    
    for question in questions:
        try:
            # Generar SQL
            sql_query = generate_sql(question)
            
            # Analizar complejidad
            complexity = "Complex" if "JOIN" in sql_query.upper() or "GROUP BY" in sql_query.upper() else "Simple"
            
            # Guardar resultados
            results.append({
                "question": question,
                "sql": sql_query,
                "complexity": complexity,
                "success": True
            })
            
            # Mostrar resultados
            print(f"Question: {question}")
            print(f"Complexity: {complexity}")
            print(f"SQL Generated:\n{sql_query}\n")
            print("-" * 50)
            
        except Exception as e:
            results.append({
                "question": question,
                "error": str(e),
                "success": False
            })
            print(f"Error with question: {question}")
            print(f"Error message: {str(e)}\n")
    
    return results

# Ejecutar pruebas para cada categoría
categories = {
    "Sales Analysis": sales_questions,
    "Inventory Analysis": inventory_questions,
    "Customer Analysis": customer_questions
}

all_results = {}
for category, questions in categories.items():
    all_results[category] = test_sql_generation(questions, category)


=== Testing Sales Analysis Questions ===

Question: Show me the total sales by product category for the last month
Complexity: Complex
SQL Generated:
```sql
SELECT p.category, SUM(oi.quantity * oi.unit_price) AS total_sales
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
JOIN products p ON oi.product_id = p.product_id
WHERE o.order_date >= DATE_SUB(CURDATE(), INTERVAL 1 MONTH)
GROUP BY p.category;
```

--------------------------------------------------
Question: Which products have had the highest profit margin this year?
Complexity: Complex
SQL Generated:
```sql
WITH order_profit AS (
    SELECT oi.product_id, 
           SUM(oi.quantity * oi.unit_price) AS revenue,
           SUM(oi.quantity * p.price) AS cost,
           SUM(oi.quantity * (p.price - oi.unit_price)) AS profit
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    JOIN orders o ON oi.order_id = o.order_id
    WHERE EXTRACT(YEAR FROM o.order_date) = EXTRACT(YEAR FROM CURRENT_DATE

In [24]:
def analyze_results(all_results):
    """
    Analizar los resultados de las pruebas
    """
    print("\n=== Results Analysis ===\n")
    
    for category, results in all_results.items():
        successful = len([r for r in results if r["success"]])
        total = len(results)
        complex_queries = len([r for r in results if r.get("complexity") == "Complex"])
        
        print(f"\nCategory: {category}")
        print(f"Success Rate: {successful}/{total} ({successful/total*100:.2f}%)")
        print(f"Complex Queries: {complex_queries}/{total}")
        print("-" * 30)

# Analizar resultados
analyze_results(all_results)


=== Results Analysis ===


Category: Sales Analysis
Success Rate: 4/4 (100.00%)
Complex Queries: 4/4
------------------------------

Category: Inventory Analysis
Success Rate: 4/4 (100.00%)
Complex Queries: 3/4
------------------------------

Category: Customer Analysis
Success Rate: 4/4 (100.00%)
Complex Queries: 4/4
------------------------------


# Laboratorio de Generación SQL con GPT-3.5 - Informe de Implementación

## Resumen Ejecutivo
Este laboratorio exploró la implementación de un sistema de generación de consultas SQL utilizando GPT-3.5-turbo. Se desarrolló una función principal para convertir preguntas en lenguaje natural a consultas SQL, probada con diferentes categorías de análisis de negocio.

## Objetivos del Laboratorio
- Desarrollar un sistema robusto de generación de SQL
- Probar diferentes tipos de consultas de negocio
- Evaluar la precisión y eficiencia del modelo
- Analizar la capacidad de manejo de diferentes complejidades

## Implementaciones

### 1. Sistema Base de Generación SQL
**Objetivo**: Crear una función base para generar consultas SQL.

**Características Implementadas**:
- Integración con OpenAI API
- Manejo de esquema de base de datos
- Control de errores
- Configuración de temperatura baja (0.1)

**Resultados**:
- Generación consistente de consultas
- Alta precisión en la interpretación
- Manejo efectivo de errores

### 2. Análisis de Ventas
**Objetivo**: Generar consultas para análisis de ventas y productos.

**Tipos de Consultas**:
- Ventas por categoría
- Márgenes de beneficio
- Rankings de clientes
- Tasas de crecimiento

**Resultados**:
- Consultas complejas generadas correctamente
- Manejo adecuado de agregaciones
- Cálculos temporales precisos

### 3. Análisis de Inventario
**Objetivo**: Crear consultas para gestión de inventario.

**Tipos de Consultas**:
- Control de stock
- Puntos de reorden
- Tiempo entre reabastecimientos
- Rotación de inventario

**Resultados**:
- Consultas precisas de inventario
- Manejo correcto de fechas
- Cálculos de métricas de inventario

### 4. Análisis de Clientes
**Objetivo**: Generar consultas para análisis de comportamiento de clientes.

**Tipos de Consultas**:
- Valor del tiempo de vida
- Frecuencia de compra
- Segmentación
- Análisis de crecimiento

**Resultados**:
- Consultas complejas de segmentación
- Cálculos precisos de métricas
- Análisis temporal efectivo

## Análisis Técnico

### Fortalezas
1. **Precisión**:
   - Alta tasa de éxito en generación
   - Consultas sintácticamente correctas
   - Interpretación precisa de requisitos

2. **Versatilidad**:
   - Manejo de múltiples tipos de análisis
   - Adaptabilidad a diferentes complejidades
   - Soporte para diversos casos de uso

3. **Robustez**:
   - Manejo efectivo de errores
   - Consistencia en resultados
   - Estabilidad del sistema

### Áreas de Mejora
1. **Optimización**:
   - Refinamiento de consultas complejas
   - Mejora en rendimiento
   - Optimización de subconsultas

2. **Limitaciones**:
   - Complejidad máxima de consultas
   - Dependencia del esquema proporcionado
   - Necesidad de validación adicional

## Lecciones Aprendidas

### Diseño del Sistema
1. Importancia del esquema de base de datos
2. Valor de la temperatura baja en generación
3. Necesidad de manejo de errores robusto

### Generación de Consultas
1. Beneficios de contexto claro
2. Importancia de ejemplos específicos
3. Valor de la consistencia en formato

### Mejores Prácticas
1. Mantener esquema actualizado
2. Validar consultas generadas
3. Documentar limitaciones conocidas

## Recomendaciones

### Mejoras Técnicas
1. Implementar validación de consultas
2. Desarrollar sistema de caché
3. Crear biblioteca de consultas comunes

### Mejoras de Proceso
1. Establecer pruebas automatizadas
2. Implementar monitoreo de rendimiento
3. Desarrollar documentación detallada

## Conclusión
El laboratorio demostró la viabilidad y efectividad de utilizar GPT-3.5 para la generación de consultas SQL. El sistema mostró alta precisión y versatilidad, con oportunidades de mejora en optimización y validación.

## Próximos Pasos
1. Expandir tipos de consultas soportadas
2. Implementar validación automática
3. Desarrollar interfaz de usuario
4. Crear sistema de feedback